In [2]:
import json
import re
import os
from glob import glob
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import quandl
import functools
import plotly.express as px
import plotly.graph_objects as go
from joblib import Parallel, delayed
import multiprocessing
from multiprocessing import Pool
from memoize.dataframe import memoize_df
from lmfit.models import SkewedGaussianModel

%matplotlib inline
pd.options.display.float_format = '{:,.4f}'.format

DARK_MODE = False
if DARK_MODE:
    plt.style.use('dark_background')
    plotly_template = 'plotly_dark'
else:
    plt.style.use('ggplot')
    plotly_template = 'ggplot2'

# 20230222_final_proj

@mpcs
@finm33550

Ethan Ho 2/22/2023

----

## Configuration & Helper Functions

The following cell contains helper functions and configuration options that I will use in this notebook.

In [40]:
def get_secrets(fp='./secrets.json'):
    """
    Reads secret values such as API keys from a JSON-formatted file at `fp`.
    """
    with open(fp, 'r') as f:
        data = json.load(f)
    return data

def get_quandl_api_key() -> str:
    """
    Returns Quandl API key stored in secrets.json.
    """
    secrets = get_secrets()
    key = secrets.get('NASTAQ_DATA_API_KEY')
    assert key, f"NASTAQ_DATA_API_KEY field in secrets.json is empty or does not exist"
    return key

def strip_str_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Given a DataFrame, strips values in columns with string or object
    dtype. I noticed that this was an issue when I saw some m_ticker values
    like "AAPL       " with trailing whitespace.
    """
    for col in df.columns:
        if pd.api.types.is_string_dtype(df[col]) or pd.api.types.is_object_dtype(df[col]):
            df[col] = df[col].str.strip()
    return df

@memoize_df(cache_dir='data/memoize', cache_lifetime_days=None)
def fetch_quandl_table(
    name, start_date, end_date, **kw
) -> pd.DataFrame:
    df = quandl.get_table(
        name,
        date={'gte': start_date, 'lte': end_date},
        api_key=get_quandl_api_key(),
        paginate=True,
        **kw
    )
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by='date', inplace=True)
    df.reset_index(inplace=True)
    return df

@memoize_df(cache_dir='data/memoize', cache_lifetime_days=None)
def fetch_quandl_quotemedia_prices(
    start_date, end_date, ticker
) -> pd.DataFrame:
    return fetch_quandl_table(
        name= 'QUOTEMEDIA/PRICES',
        start_date=start_date,
        end_date=end_date,
        ticker=ticker,
    )

@memoize_df(cache_dir='data/memoize', cache_lifetime_days=None)
def fetch_quandl_yc(
    name, start_date, end_date,
) -> pd.DataFrame:
    df = quandl.get(
        name,
        start_date=start_date,
        end_date=end_date,
        api_key=get_quandl_api_key(),
    ).reset_index().rename(columns={'Date': 'date'})
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by='date', inplace=True)
    return df

@memoize_df(cache_dir='data/memoize', cache_lifetime_days=None)
def fetch_quandl_spot(
    symbol, **kw
) -> pd.DataFrame:
    df = quandl.get(
        f'CUR/{symbol}',
        **kw
    ).reset_index().rename(columns={
        'DATE': 'date',
        'RATE': f'USD/{symbol}',
    })
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by='date', inplace=True)
    return df

def unique_index_keys(df, level=0) -> List[str]:
    return df.index.get_level_values(level=level).unique().tolist()

def get_next_day_of_week(date, day_of_week: int) -> str:
    """
    Monday = 0, Wednesday = 2
    """
    as_dt = pd.to_datetime(date)
    days_until = (day_of_week - as_dt.day_of_week) % 7
    out_dt = as_dt + pd.to_timedelta(days_until, 'D')
    return out_dt.strftime('%Y-%m-%d')

def get_standard_yc_cols(cols: List, col_prefix='') -> Dict:
    out = dict()
    for col_raw in cols:
        col = col_raw.lower()
        col = re.sub(r'-year', 'y', col)
        col = re.sub(r'-month', 'm', col)
        if col_prefix:
            col = col_prefix + '_' + col
        out[col_raw] = col
    return out

def get_yc(*args, col_prefix='', **kw):
    df = fetch_quandl_yc(*args, **kw)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
    df.rename(columns=get_standard_yc_cols(df.columns, col_prefix), inplace=True)
    return df

@functools.lru_cache()
def get_col_groups(cols) -> Dict:
    """
    get_col_groups(tuple(yc_daily.columns.tolist()))
    """
    out = dict()
    for col in cols:
        prefix, tenor_raw = col.split('_')
        tenor, unit = tenor_raw[:-1], tenor_raw[-1]
        if prefix not in out:
            out[prefix] = list()
        item = {
            'col': col,
            'country': prefix,
            'tenor': tenor,
            'unit': unit
        }
        out[prefix].append(item)
    return out

def bond_price(zcb, coupon_rate, tenor, coupon_freq):
    """
    Adapted from Zero_And_Spot_Curves.ipynb
    """
    times = np.arange(tenor, 0, step=-coupon_freq)[::-1]
    if times.shape[0] == 0:
        p = 1.0
    else:
        r = np.interp(times, zcb.index.values, zcb.values) # Linear interpolation
        p = np.exp(-tenor*r[-1]) + coupon_freq * coupon_rate * np.exp(-r*times).sum()
    return p

def get_zcb_curve(spot, coupons_per_year=4):
    """
    Adapted from Zero_And_Spot_Curves.ipynb
    """
    cpn_freq = 1 / float(coupons_per_year)
    for tenor, spot_rate in spot.items():
        if tenor > 0.001:
            times = np.arange(tenor-cpn_freq, 0, step=-cpn_freq)[::-1]
            coupon_half_yr = cpn_freq * spot_rate
            z = np.interp(times, spot.index.values, spot.values) # Linear interpolation
            preceding_coupons_val = (coupon_half_yr*np.exp(-z*times)).sum()
            # Question: tenor here needs to be 5 because all coupons at 5Y swap rate?
            # Answer: since we're only using 5Y rates, don't need to change this
            spot.loc[tenor] = -np.log((1-preceding_coupons_val)/(1+coupon_half_yr))/tenor
    
    # Calculate bond price for maturities T = 5 years and S = 5 years - 1 week
    T = 5.
    S = 4. + (51./52.)
    spot_copy = spot.copy()
    spot.loc['rt'] = bond_price(spot_copy, coupon_rate=spot.loc[5], tenor=T, coupon_freq=cpn_freq)
    spot.loc['rs'] = bond_price(spot_copy, coupon_rate=spot.loc[5], tenor=S, coupon_freq=cpn_freq)
    return spot

def get_zcb_curves(row):
    # Get groups by column prefix
    grps: Dict[List[Dict]] = get_col_groups(tuple(row.columns.tolist()))
    zcb_dict = dict()
    for cty, cols in grps.items():
        df = pd.DataFrame.from_records(cols).convert_dtypes()
        df['tenor'] = df['tenor'].astype(int)
        df.set_index(['tenor', 'unit'], inplace=True)
        
        lo_col = df.loc[(1, 'm'), 'col']
        lo = row[lo_col]
        lo.name = 0.0833
        
        try:
            mid_col = df.loc[(1, 'y'), 'col']
            mid = row[mid_col]
            mid.name = 1
        except KeyError:
            mid_col = df.loc[(12, 'm'), 'col']
            mid = row[mid_col]
            mid.name = 1
        
        hi_col = df.loc[(5, 'y'), 'col']
        hi = row[hi_col]
        hi.name = 5
        
        zcb = (pd.concat([lo, mid, hi], axis=1) / 100).apply(get_zcb_curve, axis=1)
        zcb.rename(columns={
            lo.name: f"{lo_col}_zcb",
            mid.name: f"{mid_col}_zcb",
            hi.name: f"{hi_col}_zcb",
            'rt': f"{cty}_rt",
            'rs': f"{cty}_rs",
        }, inplace=True)
        zcb_dict[cty] = zcb
    return zcb_dict

# Fetch Data

First, let's set our time indices. We choose to trade weekly on Wednesdays, and skip the week if the Wednesday falls on a holiday.

In [41]:
start_date = '1990-01-01'
end_date = '2022-12-16'

daily_idx = pd.date_range(start_date, end_date)
first_wed = get_next_day_of_week(start_date, 2)
wed_idx_w_holidays = pd.date_range(first_wed, end_date, freq='7D')
assert all(date.day_of_week == 2 for date in wed_idx_w_holidays)

wed_idx = [
    date for date in wed_idx_w_holidays
    if date not in pd.to_datetime([
        # Remove Wednesdays that fall on holidays
        '2012-12-26', '2013-12-25', '2014-01-01', '2018-12-26',
        '2019-12-25', '2020-01-01',
    ])
]
assert len(wed_idx_w_holidays) > len(wed_idx)

In [42]:
countries = {
    'USA': 'USD',
}

yc_dict = {
    country: (
        get_yc(f'YC/{country}', start_date=start_date, end_date=end_date, col_prefix=country.lower())
        .reindex(daily_idx)
        .fillna(method='ffill')
        .iloc[1:, :]
    ) for country in countries.keys()
}
yc_daily = pd.concat(yc_dict.values(), axis=1)
zcb = get_zcb_curves(yc_daily)
uszcb = zcb['usa']
uszcb

Using cache fp='data/memoize/fetch_quandl_yc_7609035_20230222.csv' to write results of function fetch_quandl_yc
Using cached call from data/memoize/fetch_quandl_yc_7609035_20230222.csv


,usa_1m_zcb,usa_1y_zcb,usa_5y_zcb,usa_rt,usa_rs
1990-01-02,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN
1990-01-06,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2022-12-12,0.1153,0.0467,0.0373,0.9972,0.9977
2022-12-13,0.1162,0.0456,0.0360,0.9972,0.9977
2022-12-14,0.1168,0.0456,0.0357,0.9972,0.9977
2022-12-15,0.1180,0.0457,0.0355,0.9972,0.9976


In [43]:
print(f"usa_1m is null up until {yc_daily[yc_daily.usa_1m.isnull()].index[-1]}")

usa_1m is null up until 2001-07-30 00:00:00


In [44]:
yc_daily.usa_1m.loc['2001-07-30':].head(5)

2001-07-30      NaN
2001-07-31   3.6700
2001-08-01   3.6500
2001-08-02   3.6500
2001-08-03   3.6300
Freq: D, Name: 0.0833, dtype: float64

US Treasuries didn't start issuing 1-month bonds until summer 2001, so I'll truncate the data before 2001-07-31.

In [56]:
zcb = uszcb.loc['2001-07-31':]
assert not zcb.isnull().any().any()

In [ ]:
px.line(
    zcb[['usa_1m_zcb', 'usa_1y_zcb', 'usa_5y_zcb']],
    template=plotly_template,
    labels={
        'value': 'Rate',
        'index': 'Date',
    },
    height=600,
)

# Scratch